In [ ]:
! pip install transformers==4.30.1
! pip install datasets
! pip install adapter-transformers==3.2.1

In [ ]:
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction

In [ ]:
import transformers
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving emails.csv to emails (1).csv


In [ ]:
import io
try:
  dataset = pd.read_csv('emails.csv')
except:
  dataset = pd.read_csv(io.BytesIO(uploaded['emails.csv']))


In [ ]:
dataset.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [ ]:
sentences = dataset.text.values
labels = dataset.spam.values

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)

LabelEncoder()

In [ ]:
y_enc = le.transform(labels)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences, y_enc, test_size=0.25, random_state=42, stratify=y_enc)

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df1 = pd.DataFrame({'sentences': list(X_train), 'labels': list(y_train)})
dataset = ds.dataset(pa.Table.from_pandas(df1).to_batches())

### convert to Huggingface dataset
hg_dataset1 = Dataset(pa.Table.from_pandas(df1))

In [ ]:
df2 = pd.DataFrame({'sentences': list(X_test), 'labels': list(y_test)})
dataset = ds.dataset(pa.Table.from_pandas(df2).to_batches())

### convert to Huggingface dataset
hg_dataset2 = Dataset(pa.Table.from_pandas(df2))

In [ ]:
import datasets
dd = datasets.DatasetDict({"train":hg_dataset1,"test":hg_dataset2})

In [ ]:
def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(
      batch["sentences"],
      max_length=256,
      truncation=True,
      padding="max_length"
  )

# Encode the input data
dd = dd.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
# dd = dd.rename_column("labels", "labels")
# Transform to pytorch tensors and only output the required columns
dd.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/4296 [00:00<?, ? examples/s]

Map:   0%|          | 0/1432 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertConfig, BertModelWithHeads

id2label = dict(list(enumerate([str(x) for x in le.classes_])))

config = BertConfig.from_pretrained(
    "bert-large-uncased",
    id2label=id2label,
)
model = BertModelWithHeads.from_pretrained(
    "bert-large-uncased",
    config=config,
)


/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/bert/adapter_model.py:269: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/bert/adapter_model.py:247: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactl

In [ ]:
model.add_classification_head("cb", num_labels=len(id2label))
model.to(device)

BertModelWithHeads(
  (shared_parameters): ModuleDict()
  (bert): BertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (key): Linear(
                in_features=1024, out_features=1024, bias=True
                (loras): ModuleDict()
              )
              (value): Linear(
                in_features=1024

In [ ]:
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=200,
    output_dir="./training_output",
    # overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dd["train"],
    eval_dataset=dd["test"],
    compute_metrics=compute_accuracy,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4296
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1076
  Number of trainable parameters = 336193538


Step,Training Loss
200,0.100400
400,0.026000
600,0.009300
800,0.005600
1000,0.000000


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/config.json
Configuration saved in ./training_output/checkpoint-500/generation_config.json
Model weights saved in ./training_output/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./training_output/checkpoint-1000
Configuration saved in ./training_output/checkpoint-1000/config.json
Configuration saved in ./training_output/checkpoint-1000/generation_config.json
Model weights saved in ./training_output/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1076, training_loss=0.02628386351600165, metrics={'train_runtime': 2466.4866, 'train_samples_per_second': 6.967, 'train_steps_per_second': 0.436, 'total_flos': 8034858071064576.0, 'train_loss': 0.02628386351600165, 'epoch': 4.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1432
  Batch size = 16


{'eval_runtime': 66.79,
 'eval_samples_per_second': 21.44,
 'eval_steps_per_second': 1.348,
 'epoch': 4.0}

In [ ]:
preds=trainer.predict(dd["test"])
(np.argmax(preds.predictions[1],axis=1) == dd["test"]["labels"].numpy()).mean()

***** Running Prediction *****
  Num examples = 1432
  Batch size = 16


0.9965083798882681

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(dd["test"]["labels"].numpy(),np.argmax(preds.predictions[1],axis=1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1090
           1       0.99      1.00      0.99       342

    accuracy                           1.00      1432
   macro avg       0.99      1.00      1.00      1432
weighted avg       1.00      1.00      1.00      1432

